# CURRENT CLICK 

## Libraries and FPGA image

We will start by importing the *rp* (Red Pitaya) and *rp_overlay* libraries, loading the *v0.94* FPGA image, and initializing the Red Pitaya.

In [ ]:
import time
from rp_overlay import overlay
import rp

# Initialize the FPGA overlay
fpga = overlay()
rp.rp_Init()

## Macros
Here is a complete list of macros that will come in handy when customising this notebook and operating the click shields.

- **States** - RP_LOW, RP_HIGH
- **Directions** - RP_IN, RP_OUT
- **LEDs** - RP_LED0, RP_LED1, ..., RP_LED7
- **DIOx_P** - RP_DIO0_P, RP_DIO1_P, ..., RP_DIO7_P &emsp;&emsp; *# Goes up to 9 on SDRlab and STEMlab 4-Input*
- **DIOx_N** - RP_DIO0_N, RP_DIO1_N, ..., RP_DIO7_N &emsp;&thinsp; *# Goes up to 9 on SDRlab and STEMlab 4-Input*
- **Analog outputs** - RP_AOUT0, RP_AOUT1, ..., RP_AOUT3
- **Analog inputs** - RP_AIN0, RP_AIN1, ..., RP_AIN3

### Click shield macros
For easier work with the click shields, here are some macro rewrites. Marcos ending in "1" are on MicroBus1, and macros ending in "2" are on MicroBus2.

- **AN1, AIN2** - RP_AIN0, RP_AIN1
- **RST1, RST2** - RP_DIO2_N, RP_DIO4_N
- **PWM1, PWM2** - RP_DIO1_P, RP_DIO3_P
- **INT1, INT2** - RP_DIO2_P, RP_DIO4_P
- **UART_SW** - RP_DIO5_N
- **SPI_CS1, SPI_CS2** - RP_DIO1_N, RP_DIO3_N
- **Logic analyser** - RP_DIO0_P, RP_DIO1_P, ..., RP_DIO7_P

Setting up click shield macros:

In [ ]:
AN1 = rp.RP_AIN0
AN2 = rp.RP_AIN1
RST1 = rp.RP_DIO2_N
RST2 = rp.RP_DIO4_N
PWM1 = rp.RP_DIO1_P
PWM2 = rp.RP_DIO3_P
INT1 = rp.RP_DIO2_P
INT2 = rp.RP_DIO4_P
UART_SW = rp.RP_DIO5_N
SPI_CS1 = rp.RP_DIO1_N
SPI_CS2 = rp.RP_DIO3_N

# Reset analog pins
rp.rp_ApinReset()

AIN = AN1

In [ ]:
def convert_to_engineering_unit(number, units):
    magnitude = number
    unit_index = 0

    while magnitude <= 1.0 and unit_index < len(units) - 1:
        magnitude *= 1000.0
        unit_index += 1

    print("Value: {:.4f} {} \n".format(magnitude, units[unit_index]))

def get_current(pin, resistor_value):
    # Get ADC value from the specified pin
    value = rp.rp_ApinGetValue(pin)[1]
    max_current = 3.3 / (20.0 * resistor_value)
    print("Minimum current is 2mA, maximum is:")
    convert_to_engineering_unit(max_current, ["A", "mA", "μA", "nA"])

    # Calculate current
    voltage = value / 20.0
    # gain of circuit is 20
    current = voltage / resistor_value

    return current

def recommend_resistor(current):
    if current != 0:
        resistor = 0.075 / current
        print("The recommended resistor value for best measurement (R) is {:.4f} ohms.\n".format(resistor))
        return resistor
    else:
        print("Error: Current (I) cannot be zero.\n")
        return -1
        # Return -1 to indicate an error

The example shows how to read the current through a shunt resistor. The program, however, does not know how many ohms your shunt resistor has. That is why the shunt resistor value must be input when prompted by the program. Also, the shunt resistor should be accurate to at least three decimals (for example, with a multimeter). That way, the current measurement will be most accurate.

Refrain from going above or below the minimum and maximum current measurement.

For best measurements, please use the recommended resistor.

In [ ]:
while True:
    print("Enter shunt resistor value in Ohms: ")
    # circuit 5 ohms
    resistor_value = float(input())
    current = get_current(AIN, resistor_value)

    convert_to_engineering_unit(current, ["A", "mA", "μA", "nA"])
    recommend_resistor(current)
    time.sleep(1)

In [ ]:
# Release resources
rp.rp_Release()